In [1]:
from datasets import load_dataset

dataset = load_dataset("diwank/samantha-toolformer-dataset")

Found cached dataset parquet (/home/diwank/.cache/huggingface/datasets/diwank___parquet/diwank--samantha-toolformer-dataset-40fc10ef8af98ee4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
remove_t = lambda text: (
    text
    .replace("toolformer: enabled\n", "")
    .replace("toolformer access:", "Tools available:")
)

def remove_toolformer(row):
    chatml = row['chatml']
    text = row['text']

    text = remove_t(text)

    chatml_idx_to_edit = [
        idx 
        for idx, s in enumerate(chatml)
        if 'toolformer' in s['content']
    ][0]

    chatml[chatml_idx_to_edit]["content"] = remove_t(
        chatml[chatml_idx_to_edit]["content"]
    )

    return dict(chatml=chatml, text=text)

In [7]:
dataset = dataset.map(remove_toolformer)

Map:   0%|          | 0/7673 [00:00<?, ? examples/s]

In [8]:
dataset['train'][0]['chatml']

[{'content': '09 November 2020\n\nSamantha is talking to Lawrence. Samantha has the following tools available if she needs them: wikipedia',
  'name': 'situation',
  'role': 'system'},
 {'content': 'What was the cause of the French Revolution?',
  'name': 'Lawrence',
  'role': 'user'},
 {'content': 'This needs additional information. I need to use the tools in order to answer this.',
  'name': 'thought',
  'role': 'system'},
 {'content': 'Tools available: wikipedia\nA wrapper around Wikipedia. Useful for when you need to display general information about people, places, companies, historical events, or other subjects found in an encyclopedia, displays a snippet summary of the topic. Input should be a search query.\nwikipedia(query)\nProvide a summary of a famous historical event based on the input event given',
  'name': 'information',
  'role': 'system'},
 {'content': "Summary of the French Revolution: wikipedia('French Revolution')",
  'name': 'thought',
  'role': 'system'}]

In [9]:
dataset.push_to_hub("diwank/samantha-toolformer-dataset", private=True)

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]